 # MODELE AVEC PLUSIEURS GROUPES


   
Dans ce modèle nous allons chercher, en nous inspirant du modèle de Nicky Case, à générer plusieurs groupes en même temps et ensuite les relier entre elles.

Tout d'abord nous reprenons des fonctions du modèle simple pour les utiliser

In [1]:
import math
import random
def nbr_amis(personne):
    """list[int]->int
    renvoie le nombre d'amis d'une personne à partir d'une liste de 0 et de 1"""
    nbr=0
    for elt in personne:
        if elt==1:
            nbr+=1
    return nbr

def choisir_amis(no_personne,n,nbr_moy):
    """int*int->list[int]
    renvoie les une liste avec les amis choisis d'une personne"""
    LR=[]
    exclusion= [i for i in range(n) if i!=no_personne]
    last=0
    for elt in range(nbr_moy):
        last=random.choice(exclusion)
        LR.append(last)
        exclusion.remove(last)
    return LR
        
        
            
def donneur_info(population,donneur):
    """int*int->list[int]
    donne les matrices info et stock_info"""
    info=[0 if i!= donneur else 1 for i in range(population)]
    return info


ProfilTypes= {"classique":0.3,"marginal":0.3,"vieux":0.1,"geek":0.2,"media":0.1}
ProfilSpecs={"classique":(2/13,0.3,0.7,0.02),
             "marginal":(1/15,0.1,0.4, 0.01),
             "vieux":(1/13,0.6,0.9,0.005),
             "geek":(4/13,0.7,1,0.07),
            "media":(30/100,0.2,0.8,0.1)} #format nom:(proportion d'amis, proba d'envoyer, proba de recevoir)

def listprofil(probaprofil,n):
    """dict[str:int]*int->list[str]
    renvoie une liste de n personnes avec leur profil"""
    LR=[]
    nbr=[]
    for i in probaprofil.keys():
        nbr=nbr+[i for n in range(round(n*probaprofil[i]))]
    if len(nbr) != n:
        for i in range(n-len(nbr)):
            nbr.append(random.choice([i for i in probaprofil.keys()]))
    for i in range(n):
        a=random.choice(nbr)
        LR.append(a)
        nbr.remove(a)

    return LR

def simple_graph_generator_profils(listprofils, profilspecs):
    """list[str]*dict[str:tuple[int;int;int]]->list[list[int]]
    renvoie une matrice liaison en tenant compte des différents profils"""
    n=len(listprofils)
    profils=list(profilspecs.keys())
    LR=[[0 for elt in range(n)] for i in range(n)]
    nbr_moy=dict((i,round(n*ProfilSpecs[i][0])) for i in profils) 
    for elt in range(n):
        amis=choisir_amis(elt,n,nbr_moy[listprofils[elt]])
        for elt2 in amis:
            tolerance=round(n*1/10)
            if nbr_amis(LR[elt]) <= nbr_moy[listprofils[elt]]+tolerance and nbr_amis(LR[elt2]) <= nbr_moy[listprofils[elt2]]+tolerance:
                LR[elt][elt2]=1
                LR[elt2][elt]=1
    return LR


Ici nous créons une fonction groupe_generator qui nous permet de générer plusieurs groupes avec des tailles et des proportions de types différents.
Cette fonction nous retourne:
- un dictionnaire avec la pour chaque groupe la matrice liaison
- un dictionnaire avec pour chaque type le nombre d'individue dans chaque groupe
- un dictionnaire avec la lilste des profils des chaque groupes

(les fonctions list_to_dico et organisation_dico serve juste à remettre en ordre nos listes et dictionnaires)

In [2]:
def list_to_dico(l):
    """list[alpha]->dict[alpha:int]
    apprends a lire connard"""
    DR=dict()
    for i in l:
        if i not in DR:
            DR[i]=1
        else:
            DR[i]+=1
    return DR

def organisation_dico(dico):
    """dict[int:dict[str:int]]->dict[str:list[int]]
    transforme le dico propre pour la suite"""
    D=dict((i,[0 for i in range(len(dico))]) for i in ProfilTypes)
    for (k,v) in dico.items():
        for (i,j) in v.items():
            D[i][k]=j
    return D

import string
def groupe_generator(profilspecs,probaprofil,taille):
    """int*list[int] -> dict[str:list[list[int]]]
    Génere n groupes de certaine taille"""
    profils=list(ProfilSpecs.keys())
    n=len(taille)
    DR=dict()
    inventairetemp=dict((i,dict((i,0)for i in profils)) for i in range(n))
    listprofils=[]
    Dico_types=dict()
    for elt in range(n):
        
        Dico_types[elt]=list(listprofil(probaprofil[elt],taille[elt]))
        DR[elt]=simple_graph_generator_profils(Dico_types[elt],profilspecs)
        inventairetemp[elt]=list_to_dico(Dico_types[elt])
    return DR,organisation_dico(inventairetemp),Dico_types

ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
print(groupe_generator(ProfilSpecs,ProbaProfil,[10,10,10,10]))


({0: [[0, 1, 1, 1, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 1, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 1, 1, 1], [1, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 1, 0, 0, 1, 0, 1, 0, 0, 0]], 1: [[0, 0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0, 0]], 2: [[0, 0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0]], 3: [[0, 0, 0, 0, 1, 0, 

La fonction nbr_liaisons_entre_groupes nous permet de savoir le nombre de liaisons que chaque groupe va faire en fonction des probabilité de chaque type de faire des liaisons ponts.
cette fonction nous retourne:
- un dictionnaire avec pour chaque type le nombre de personne de chaque groupe qui fait une liaison pont

(on peux noter que l'on sait donc le type de la personne qui fait la liaison. Mais aussi que connait le type mais pas la personne ce qui veux dire que l'on peux avoir par exemple 2 geek qui font des liaisons ponts ou un geek qui fait 2 liaisons ponts)

In [3]:

def nbr_liaisons_entre_groupes(Dtype,profilspecs):
    """dict[str:list[int]]*dict[str:list[float]] -> dict[str:list[int]]
    renvoie un dictionnaire qui associe à chaque type les gens de chaque groupe qui font des liaisons avec un autre groupe
    """
    n=len(Dtype['classique'])
    
    DR=dict((i,[0 for i in range(n)]) for i in ProfilTypes)
    ii=0
    for (i,j) in Dtype.items():
        
        for elt in j:
            
            for elt2 in range(elt):
                rdm=random.random()
                if rdm<profilspecs[i][3]:
                    
                    DR[i][ii]+=1
                            
            ii+=1
        ii=0                  
            
            
            
        
    return DR

ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
A=groupe_generator(ProfilSpecs,ProbaProfil,[5,4,9,8,6])[1]
print(A)
print(nbr_liaisons_entre_groupes(A,ProfilSpecs))

{'classique': [0, 3, 0, 3, 2], 'geek': [4, 0, 4, 3, 2], 'marginal': [0, 0, 2, 0, 0], 'media': [1, 0, 1, 1, 1], 'vieux': [0, 1, 2, 1, 1]}
{'classique': [0, 0, 0, 0, 0], 'geek': [1, 0, 0, 0, 0], 'marginal': [0, 0, 0, 0, 0], 'media': [0, 0, 0, 0, 0], 'vieux': [0, 0, 0, 0, 0]}


la fonction type_liaisons nous retourne le type de l'amis avec qui la personne en question fait une liaison pont

la fonction choisir_groupe nous retourne le groupe dans lequel se situe cette amis

In [4]:
ProfilSpecs={"classique":(2/13,0.3,0.7,0.2),
             "marginal":(1/15,0.1,0.4, 0.1),
             "vieux":(1/13,0.6,0.9,0.05),
             "geek":(4/13,0.7,1,0.7),
            "media":(30/100,0.2,0.8,0.1)}
ListType=["classique","marginal","vieux","geek","media"]


def type_liaisons(Type,listtype):
    """str -> str
    renvoie, en fonction du type de qqn, le type de la personne avec qui elle se connecte"""
    T=''
    rdm1=random.random()
    L=list(listtype)
    L.remove(Type)
    rdm2=random.choice(L) #à regarder si on peux mettre un choices pour mettre des poids
    if rdm1<0.4:
        T=Type
    else:
        T=rdm2
        
    return T

T=type_liaisons("geek",ListType)
print(T)

ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
A=groupe_generator(ProfilSpecs,ProbaProfil,[5,4,9,8,6])[1]
print(A[T])

def choisir_groupe(typeL,groupeP,Dtype):
    """str*int*dict[str:list[int] -> str
    renvoie le groupe avec lequel la personne va effectuer une liaisons en fonction du type 
    de la personne avec qui elle effectue la liaisons"""
    L=list(Dtype[typeL])
    L[groupeP]=0
    nbrT=0
    P=[]
    rdm=random.random()
    temp=0
    for i in L:
        nbrT+=i
    for i in L:        
        P.append(temp+i/nbrT)
        temp+=i/nbrT
    G=0
    

    while rdm>P[G]:
        G+=1
        
    return G
    

g=choisir_groupe(T,0,A)
print(g)

geek
[5, 0, 4, 3, 2]
3


la fonction liaisons_groupe va nous créer, grâce aux fonctions précédente,

In [8]:
def liaisons_groupe(nbr_liaisons,Dtype):
    """dict[str:list[int]]**2 -> list[list[int]]*dict[int:dict[int:list[tuple[tuple[str,int],tuple[str,int]]]]
    créé une matrice liaison entre groupes"""
    
    n=len(Dtype['classique'])
    LR=[[0 for elt in range(n)] for i in range(n)]
    
    typetemp=''
    grouptemp=42
    
    DR=dict((a,dict()) for a in range(n))

    for (i,j) in nbr_liaisons.items():
        gpelt1=-1
        for elt1 in j:
            gpelt1+=1
            for elt2 in range(elt1):
                
                typetemp=type_liaisons(i,[s for s in nbr_liaisons])
                grouptemp=choisir_groupe(typetemp,gpelt1,Dtype)
                
                L1=[a for a in range(Dtype[i][gpelt1])]
                L2=[a for a in range(Dtype[typetemp][grouptemp])]
                
                rdm1=random.choice(L1)
                rdm2=random.choice(L2)
                
                #print(i,rdm1,'du groupe',gpelt1, 'fait une liaison avec', typetemp,rdm2,'du groupe', grouptemp)
                
                if gpelt1 not in DR[grouptemp]:
                    DR[gpelt1][(i,rdm1)]=[(grouptemp,(typetemp,rdm2))]
                    DR[grouptemp][(typetemp,rdm2)]=[(gpelt1,(i,rdm1))]
                else:
                    DR[gpelt1][(i,rdm1)].append((grouptemp,(typetemp,rdm2)))
                    DR[grouptemp][(typetemp,rdm2)].append((gpelt1,(i,rdm1)))
                    
                LR[gpelt1][grouptemp]+=1
                LR[grouptemp][gpelt1]+=1
                    
                    
                
    return LR,DR

ProfilSpecs={"classique":(2/13,0.3,0.7,1),
             "marginal":(1/15,0.1,0.4, 1),
             "vieux":(1/13,0.6,0.9,1),
             "geek":(4/13,0.7,1,1),
            "media":(30/100,0.2,0.8,0.10)}



ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
A=groupe_generator(ProfilSpecs,ProbaProfil,[5,4,9,8,6])[1]
B=nbr_liaisons_entre_groupes(A,ProfilSpecs)
print("nombre de liaisons entre groupes:",B)
print(A)    
L=liaisons_groupe(B,A)
print(L)
                    
                      

nombre de liaisons entre groupes: {'classique': [0, 3, 0, 3, 2], 'geek': [4, 0, 4, 3, 2], 'marginal': [1, 0, 2, 0, 0], 'media': [0, 0, 0, 0, 0], 'vieux': [0, 1, 2, 1, 1]}
{'classique': [0, 3, 0, 3, 2], 'geek': [4, 0, 4, 3, 2], 'marginal': [1, 0, 2, 0, 0], 'media': [0, 0, 1, 1, 1], 'vieux': [0, 1, 2, 1, 1]}
([[0, 0, 4, 3, 1], [0, 0, 4, 2, 3], [4, 4, 0, 4, 3], [3, 2, 4, 0, 5], [1, 3, 3, 5, 0]], {0: {('geek', 0): [(3, ('geek', 0))], ('marginal', 0): [(2, ('marginal', 0))], ('geek', 1): [(3, ('geek', 0))], ('geek', 2): [(2, ('media', 0))]}, 1: {('classique', 1): [(4, ('geek', 0))], ('classique', 2): [(3, ('classique', 2))], ('vieux', 0): [(2, ('marginal', 1))], ('classique', 0): [(2, ('vieux', 0))]}, 2: {('marginal', 1): [(1, ('vieux', 0))], ('marginal', 0): [(4, ('vieux', 0))], ('geek', 0): [(0, ('marginal', 0))], ('vieux', 0): [(1, ('classique', 0))], ('vieux', 1): [(4, ('media', 0))], ('geek', 3): [(3, ('geek', 1))], ('media', 0): [(0, ('marginal', 0))], ('geek', 2): [(1, ('classique', 

In [9]:
def passage_info_simple_profils(liaisons,info,stock_info,nbr,profilspecs,listeprofils):
    """list[list[int]]*list[int]*list[int]*int->list[int]
    Transmet l'information à la géneration suivante"""
    new_info=[0 for i in info]
    new_stock_info=[i for i in stock_info]
    nbr_new=nbr
    for i in range(len(info)):
        if info[i]==1:
            for a in range(len(liaisons[i])):
                rdm=random.random()
                if liaisons[i][a]==1 and rdm<profilspecs[listeprofils[i]][1]: #a modifier pour modèle complexe
                    if new_stock_info[a]<1 and rdm<profilspecs[listeprofils[i]][2]: #a modifier pour modèle complexe
                        new_info[a]=1
                        nbr_new+=1
                        new_stock_info[a]=1
                    #elif rdm<profilspecs[listeprofils[i]][2]:
                        #new_stock_info[a]+=1
                
    return new_info,new_stock_info,nbr_new


In [10]:
group_info=[1,0,0,0,0]
Dico_ListeProfilTest=dict((i,listprofil(ProfilTypes,10))for i in range(10))
#print(Dico_ListeProfilTest)
def position_type(dico_listeprofils):
    """dict[int:list[str]] -> dict[int:list[tuple[str,int]]]
    renvoie un dictionnaire avec pour chaque groupe les personnes en fonctions de leur type et leur numéro
    (le numéro correspond au combientième du groupe il est de son type)"""
    
    new_dico_listeprofils=dict((i,[[str,0]for a in range(len(dico_listeprofils[i]))]) for i in dico_listeprofils)
    
    for i in dico_listeprofils:
        DRtypes={'vieux':0,'classique':0,'marginal':0,'geek':0}
        for y in range(len(dico_listeprofils[i])):
            for z in DRtypes:
                if z==dico_listeprofils[i][y]:

                        new_dico_listeprofils[i][y]=[z,DRtypes[z]]
                        DRtypes[z]+=1
    
    return new_dico_listeprofils

#position_type(Dico_ListeProfilTest)

def position_fct_type_num(groupe,personne,dico_listeprofils):
    """
    
    retrouve la position d'une personne dans un groupe en fonction de son type et de son numéro"""
    
    num=42
    cpt=0
    for i in range(len(dico_listeprofils[groupe])):
        #print(dico_listeprofils[groupe][i])
        if dico_listeprofils[groupe][i]==personne[0]:
            #print("coucou")
            if cpt==personne[1]:
                num=i
                return num
            else:
                cpt+=1
                

In [11]:
def transmisions_info_groupes(dico_liaisons,dico_info,specs_groupes, \
                              dico_stock_info,dico_nbr,profilspecs, dico_listeprofils_num,dico_listeprofils):
    """dict[str:list[list[int]]]*list[int]*dict[int:dict[int:tuple[tuple[str,int],tuple[str,int]]]] ->
    -> 
    
    dico_groupes: dictionnaires des différentes liaisons dans chaque groupe
    dico_info: dictionnaire des personnes possédant l'informations dans chaque groupe
    specs_groupes: dictionnaire des liaisons entre groupes avec types et numéros des personnes faisant les liaisons
    group_info: liste des groupe qui possède l'info
    dico_stock_info: dictionnaire du nombre de fois que chaque personnes de chaque groupes à recus l'informations
    dico_nbr: nbr de personnes possédant l'information dans chaque groupe
    profilspecs: donnés sur les différents types
    dico_listeprofils_num: dictionnaire de chaque groupe indiquant le type et le numéro de chaque personne
    
    renvoie qui à l'information après une génération dans les différents groupes"""
    
    D_new=dict(dico_info)
    #print("D_new est égale à",D_new)
    Dstock_new=dict(dico_stock_info)
    #print(Dstock_new)
    Dnbr_new=dict(dico_nbr)
    ngroup=len(group_info)
    for i in range(ngroup): #i parcour les différents groupes
        if dico_nbr[i]!=0: #vérifie si le groupe i possède l'information
            
            a,b,c=passage_info_simple_profils(dico_liaisons[i],dico_info[i],dico_stock_info[i],\
                                              dico_nbr[i],profilspecs,dico_listeprofils[i])
            
            #print('liaisons du groupe',i,'---',dico_liaisons[i])
            print('nouvelle info, stock info et nbr du groupe',i,"---",a,b,c)
            D_new[i]=a
            Dstock_new[i]=b
            #print(Dstock_new)
            Dnbr_new[i]=c
            #print(a,b,c)
            #print(Dnbr_new)
            for elt1 in range(len(D_new[i])): #elt1 parcour les personnes du groupes i
                #print(elt1)
            
            
                if dico_info[i][elt1]==1:  #vérifie que la personne elt1 possède l'information
                    #print(dico_listeprofils_num[i][elt1])
                    #print(specs_groupes[i])
                    A,B=(dico_listeprofils_num[i][elt1]) #détermine le type et le numéro de la personne
                    #print(A,B)
                    
                    if (A,B) in specs_groupes[i]: #vérifie que la personne fasse une liaison avec un autre groupe
                        #print("coucou",specs_groupes)
                        #print(specs_groupes[i])
                        #print(specs_groupes[i][A,B])
                        for I in range(len(specs_groupes[i][A,B])): #fait une boucle pour chaque amis dans un autre groupe
                            groupamis=specs_groupes[i][A,B][I][0] #détermine le groupe de l'amis
                            #print("groupamis:",groupamis)
                            amis=specs_groupes[i][A,B][I][1] #détermine le type et le numéro de l'amis
                            #print("amis:",amis) 
                            position_amis=position_fct_type_num(groupamis,amis,dico_listeprofils) #détermine la position de l'amis
                            #print("position amis est",position_amis)
                       
                            rdm=random.random()
                            #print(amis[1])
                            if rdm<profilspecs[A][1]: #vérifie que elt1 envoie bien l'information
                                if Dstock_new[groupamis][position_amis]<1 and rdm<profilspecs[amis[0]][2]: 
                                    #vérifie que l'amis reçoit l'information et qu'il n'avait pas déja l'information
                                    print("coucou")
                                    D_new[groupamis][position_amis]=1 #il gagne l'info
                                    Dnbr_new[groupamis]+=1 #une personne en plus possède l'info
                                    Dstock_new[groupamis][position_amis]=1 #il à reçus une fois l'info
                                elif rdm<profilspecs[amis[0]][2]:
                                    #si il reçoit l'info mais qu'il l'avait déjà on augmente juste stock_info
                                    Dstock_new[groupamis][position_amis]+=1
                            
                    
            
    return D_new,Dstock_new,Dnbr_new


def define_dico_info(groupe,personne,taille):
    """"""
    DR=dict((elt,[0 for i in range(taille[elt])]) for elt in range(len(taille)))
    DRcopy=dict(DR)
    for i in DRcopy:
        if i == groupe:
            for ii in range(len(DRcopy[i])):
                if ii==personne:
                    DR[i][ii]=1
    return DR
        
def Define_Dico_nbr(taille_groupes):
    """"""
    Dico_nbr=dict((i,0) for i in range(1,len(taille_groupes)))
    Dico_nbr[0]=1
    
    return Dico_nbr


ProfilSpecs={"classique":(2/13,1,1,0.2),
             "marginal":(1/15,1,1, 0.1),
             "vieux":(1/13,1,1,0.05),
             "geek":(4/13,1,1,0.7),
            "media":(15/100,0.8,0.8,0.2)}
    
taille_groupes=[20,24,29,28,26]
ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
Dico_Liaisons=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[0]

Dico_Info=define_dico_info(0,1,taille_groupes)


A=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[1]

B=nbr_liaisons_entre_groupes(A,ProfilSpecs)
#print(A,B)
Specs_Groupes=liaisons_groupe(B,A)[1]
#print(Specs_Groupes)

Dico_stock_info=Dico_Info
Dico_nbr=Define_Dico_nbr(taille_groupes)
Dico_ListeProfil=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[2]
print(Dico_ListeProfil)
Dico_listeprofils_num=position_type(Dico_ListeProfil)
#print(Dico_listeprofils_num)

print(position_fct_type_num(0,('classique',0),Dico_ListeProfil))       
print(transmisions_info_groupes(Dico_Liaisons,Dico_Info,Specs_Groupes, \
                              Dico_stock_info,Dico_nbr,ProfilSpecs,Dico_listeprofils_num,Dico_ListeProfil))        
	

            

{0: ['geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'media', 'geek', 'media', 'geek', 'geek'], 1: ['media', 'classique', 'classique', 'marginal', 'media', 'classique', 'classique', 'classique', 'vieux', 'vieux', 'classique', 'classique', 'classique', 'classique', 'marginal', 'classique', 'classique', 'classique', 'classique', 'classique', 'classique', 'classique', 'classique', 'marginal'], 2: ['media', 'geek', 'geek', 'marginal', 'geek', 'geek', 'vieux', 'geek', 'vieux', 'geek', 'geek', 'vieux', 'marginal', 'geek', 'media', 'geek', 'media', 'marginal', 'vieux', 'geek', 'geek', 'geek', 'vieux', 'vieux', 'marginal', 'geek', 'marginal', 'marginal', 'geek'], 3: ['media', 'media', 'vieux', 'classique', 'classique', 'geek', 'classique', 'classique', 'classique', 'classique', 'geek', 'geek', 'classique', 'geek', 'vieux', 'classique', 'classique', 'media', 'geek', 'geek', 'geek', 'geek', 'geek', 'vieux', 'geek', 'geek', '

In [12]:
def transmisions_info_groupes_n(dico_liaisons,dico_info,specs_groupes, \
                              dico_stock_info,dico_nbr,profilspecs, dico_listeprofils_num,dico_listeprofils,n):
    """"""
    
    dico_info_new=dico_info
    dico_stock_info_new=dico_stock_info
    dico_nbr_new=dico_nbr
        
    grosse_matrice_info_n=[grosse_matrice(dico_info)]
    grosse_matrice_stock_info_n=[grosse_matrice(dico_stock_info)]
    for i in range(n):
        

        dico_info_temp=dico_info_new
        dico_stock_info_temp=dico_stock_info_new
        dico_nbr_temp=dico_nbr_new
        print('génération',i,dico_nbr_temp)
        
        
        tuple_transmit=transmisions_info_groupes(dico_liaisons,dico_info_temp,specs_groupes, \
                              dico_stock_info_temp,dico_nbr_temp,profilspecs,dico_listeprofils_num,dico_listeprofils)
        dico_info_new=tuple_transmit[0]
        dico_stock_info_new=tuple_transmit[1]
        dico_nbr_new=tuple_transmit[2]
        
        grosse_matrice_info=grosse_matrice(dico_info_new)
        grosse_matrice_stock_info=grosse_matrice(dico_stock_info_new)
        
        grosse_matrice_info_n.append(grosse_matrice_info)
        grosse_matrice_stock_info_n.append(grosse_matrice_stock_info)
        
    return dico_info_new,dico_stock_info_new,dico_nbr_new,grosse_matrice_info_n,grosse_matrice_stock_info_n

ProfilSpecs={"classique":(2/13,1,1,0.2),
             "marginal":(1/15,1,1, 0.1),
             "vieux":(1/13,1,1,0.05),
             "geek":(4/13,1,1,0.7),
            "media":(15/100,0.8,0.8,0.2)}
    
taille_groupes=[20,24,29,28,26]
ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1}}
Dico_Liaisons=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[0]

Dico_Info=define_dico_info(0,1,taille_groupes)


A=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[1]

B=nbr_liaisons_entre_groupes(A,ProfilSpecs)
#print(A,B)
Specs_Groupes=liaisons_groupe(B,A)[1]
#print(Specs_Groupes)

Dico_stock_info=Dico_Info
Dico_nbr=Define_Dico_nbr(taille_groupes)
Dico_ListeProfil=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[2]
print(Dico_ListeProfil)
Dico_listeprofils_num=position_type(Dico_ListeProfil)
#print(Dico_listeprofils_num)

print(position_fct_type_num(0,('classique',0),Dico_ListeProfil))       
print(transmisions_info_groupes(Dico_Liaisons,Dico_Info,Specs_Groupes, \
                              Dico_stock_info,Dico_nbr,ProfilSpecs,Dico_listeprofils_num,Dico_ListeProfil))        
	

n=50
print(Dico_Liaisons)
#print(position_fct_type_num(0,('classique',0),Dico_ListeProfil))       
print(transmisions_info_groupes_n(Dico_Liaisons,Dico_Info,Specs_Groupes, \
                              Dico_stock_info,Dico_nbr,ProfilSpecs,Dico_listeprofils_num,Dico_ListeProfil,n)[3]) 
print(transmisions_info_groupes_n(Dico_Liaisons,Dico_Info,Specs_Groupes, \
                              Dico_stock_info,Dico_nbr,ProfilSpecs,Dico_listeprofils_num,Dico_ListeProfil,n)[4]) 
	

            

{0: ['geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'geek', 'media', 'geek', 'geek', 'media', 'geek'], 1: ['classique', 'classique', 'classique', 'classique', 'vieux', 'classique', 'classique', 'classique', 'classique', 'marginal', 'marginal', 'classique', 'classique', 'vieux', 'media', 'media', 'classique', 'classique', 'classique', 'vieux', 'classique', 'classique', 'classique', 'classique'], 2: ['media', 'geek', 'geek', 'vieux', 'geek', 'vieux', 'marginal', 'geek', 'media', 'geek', 'vieux', 'marginal', 'geek', 'marginal', 'geek', 'media', 'geek', 'geek', 'vieux', 'geek', 'geek', 'geek', 'marginal', 'vieux', 'marginal', 'vieux', 'geek', 'marginal', 'geek'], 3: ['media', 'geek', 'geek', 'geek', 'classique', 'classique', 'classique', 'classique', 'classique', 'vieux', 'geek', 'media', 'classique', 'classique', 'classique', 'geek', 'classique', 'geek', 'classique', 'geek', 'geek', 'geek', 'geek', 'vieux', 'vieux', 'media',

NameError: name 'grosse_matrice' is not defined

In [26]:
Dico_test={0: [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

def grosse_matrice(Dico_info):
    """"""
    grosse_matrice=[]
    
    for a,elt1 in Dico_info.items():
        #print(elt1)
        for elt2 in elt1:
            #print(elt2)
            grosse_matrice.append(elt2)
            
    return grosse_matrice

print(grosse_matrice(Dico_test))


def grosse_matrice_liaison(Dico_liaison,Taille_groupes,dico_listeprofils_num,dico_listeprofils,specs_groupes):
    """
    retourne une grande matrice donnant les liaisons de toute les personnes quel que soit leur groupe"""
    
    List_num=[]
    
    nbr_personne=0
    compteur=0
    T=[0]
    for i in Taille_groupes:
        nbr_personne+=i
        T.append(nbr_personne)

    nbr_groupes=len(Dico_liaison)
    dernier=T[nbr_groupes]
    T.remove(dernier)
    grosse_matrice=[[0 for i in range(nbr_personne)] for i in range(nbr_personne)]   

    for a,G1 in Dico_liaison.items():
        #print("Taille_groupes[a]:",Taille_groupes[a])
        #print("nbr_personne-Taille_groupes[a]:",nbr_personne-Taille_groupes[a])
        #print('a:',a)
        #print("type a:",type(a)," , type G1: ", type(G1))
        #print("G1:",G1)
        for P1_index in range(len(G1)):
            #print(P1_index)
            #print("type P1:",type(L_P1))
            #print("range(len(G1[P1_index])",range(len(G1[P1_index])))
            for P2_index in range(len(G1[P1_index])):
                if G1[P1_index][P2_index]==1:
                    grosse_matrice[P1_index+T[a]][P2_index+T[a]]=1
                    grosse_matrice[P2_index+T[a]][P1_index+T[a]]=1

            A,B=(dico_listeprofils_num[a][P1_index])
            if (A,B) in specs_groupes[a]:                
                for I in range(len(specs_groupes[a][A,B])):
                    groupamis=specs_groupes[a][A,B][I][0] 
                    amis=specs_groupes[a][A,B][I][1] 
                    position_amis=position_fct_type_num(groupamis,amis,dico_listeprofils)
                    
                    grosse_matrice[P1_index+T[a]][position_amis+T[groupamis]]=1
                    grosse_matrice[position_amis+T[groupamis]][P1_index+T[a]]=1
               
    return grosse_matrice


ProfilSpecs={"classique":(1/20,0.2,0.7,0.09),
             "marginal":(1/22,0.3,0.4,0.09),
             "vieux":(1/18,0.4,0.7,0.07),
             "geek":(1/17,0.5,0.8,0.12),
            "media":(20/100,0.2,0.8,0.10)} 
ProbaProfil={0:{"classique":0.0,"marginal":0.0,"vieux":0.0,"geek":0.9,"media":0.1},
             1:{"classique":0.7,"marginal":0.1,"vieux":0.1,"geek":0.0,"media":0.1},
             2:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1},
             3:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             4:{"classique":0.4,"marginal":0.0,"vieux":0.1,"geek":0.4,"media":0.1},
             5:{"classique":0.0,"marginal":0.2,"vieux":0.2,"geek":0.5,"media":0.1}}
Dico_Liaisons=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[0]
#print(Dico_Liaisons)
taille_groupes=[110,80,90,64,84,45]
Dico_listeprofils_num=position_type(Dico_ListeProfil)
Dico_ListeProfil=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[2]
A=groupe_generator(ProfilSpecs,ProbaProfil,taille_groupes)[1]

B=nbr_liaisons_entre_groupes(A,ProfilSpecs)
Specs_Groupes=liaisons_groupe(B,A)[1]


listep=grosse_matrice(Dico_ListeProfil)
mat=grosse_matrice_liaison(Dico_Liaisons,taille_groupes,Dico_listeprofils_num,Dico_ListeProfil,Specs_Groupes)


[1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [27]:
def draw_liaisons_profils(l,listeprofils):
    """list[list[int]]->NoneType
    renvoi le dessin des liaisons à partir de la matrice liaisons"""
    G=nx.Graph()
    i=len(l)
    G.add_nodes_from([i for i in range(i)])
    for elt in range(len(l)):
        for elt2 in range(len(l[elt])):
            if l[elt][elt2]==1:
                G.add_edge(elt,elt2)
    pos = nx.spring_layout(G)
    #nx.draw_networkx_nodes(G, pos=pos, node_color=node_colors, nodesize=100, alpha = 0.7)
    #nx.draw_networkx_edges(G, pos=pos)
    return G


In [28]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio
import os
import numpy as np            
import plotly.plotly as py
import networkx as nx

def clean_images(dossier):
    """str->None
    supprime les images d'un fichier"""
    for filename in os.listdir(dossier):
        if filename.endswith('.png'):
            path=str(dossier+"/"+filename)
            os.unlink(path)
    return None

ImportError: No module named 'plotly'

In [16]:
listep=grosse_matrice(Dico_ListeProfil)
mat=grosse_matrice_liaison(Dico_Liaisons,taille_groupes,Dico_listeprofils_num,Dico_ListeProfil,Specs_Groupes)
def graph_propre_profils(liaisons,listeprof):
    G=draw_liaisons_profils(liaisons,listeprof)
    pos=nx.spring_layout(G)
    clean_images("images")
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlOrRd',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Amis',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))
    print(G.nodes)
    for node in pos:
        x = pos[node][0]
        y = pos[node][1]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])


    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color']+=tuple([len(adjacencies[1])])
        node_info ='Type: ' + listeprof[node]+' , # of friends: '+str(len(adjacencies[1]))
        node_trace['text']+=tuple([node_info])

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='<br>Représentation d\'un Réseau d\'amis',
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="Population de personnes utilisant les réseaux sociaux et leur liaisons entre eux.",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    if not os.path.exists('images'):
        os.mkdir('images')
    pio.write_image(fig, 'images/fig1.png')

    return py.iplot(fig, filename='networkx')
graph_propre_profils(mat, listep)

NameError: name 'nx' is not defined

In [17]:
def stock_evolution_profils(liaisons,info,stock_info,nbr,profilspecs, listeprofils, n):
    """list[list[int]]*list[int]*list[int]*int*int->list[list[int]]
   renvoie la liste des personnes ayant eu l'information depuis le début"""
    Ltemp=[i for i in info]
    stocktemp=[i for i in stock_info]
    listock=list([list(stocktemp)])
    nbrtemp=nbr
    NBR=[1]
    for i in range(n):
        temp=passage_info_simple_profils(liaisons,Ltemp,stocktemp,nbrtemp, ProfilSpecs, listeprofils)
        Ltemp=[i for i in temp[0]]
        stocktemp=[i for i in temp[1]]
        listock.append(stocktemp)
        nbrtemp=temp[2]
    return listock
Profiltypes= {"classique":0.4, "marginal":0.2,"vieux":0.2,"geek":0.2}
ProfilSpecs={"classique":(1/19,0.6,0.6),
             "marginal":(1/21,0.4,0.4),
             "vieux":(1/18,0.5,0.7),
             "geek":(1/17,0.6,0.8),
            "media": (15/100,0.6,0.7)}
listeprofils=listprofil(Profiltypes, 50)
generate=simple_graph_generator_profils(listeprofils, ProfilSpecs)
donneur=donneur_info(50,listeprofils.index("geek"))
stock=[i for i in donneur]
listevo=stock_evolution_profils(generate,donneur,stock,1,ProfilSpecs, listeprofils, 10)
print(listevo)

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0], [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,

In [18]:
def list_to_color(liste):
    """"""
    color=[]
    for i in liste:
        if i ==1:
            color.append("red")
        else:
            color.append("white")
    return color

def clean_images(dossier):
    """str->None
    supprime les images d'un fichier"""
    for filename in os.listdir(dossier):
        if filename.endswith('.png'):
            path=str(dossier+"/"+filename)
            os.unlink(path)
    return None


In [19]:
def graph_propre_profils_transmit_groupe(liaisons,listeprof,gen):

    n=len(liaisons)
    donneur=donneur_info(n,listeprof.index("classique"))
    stock=[i for i in donneur]
    listevo=stock_evolution_profils(liaisons,donneur,stock,1,ProfilSpecs, listeprof, gen)
    clean_images("images")
    G=draw_liaisons_profils(liaisons,listeprof)
    pos=nx.spring_layout(G)
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')
    
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
    for elt in range(len(listevo)):

        node_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=False,
                # colorscale options
                #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
                #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
                #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
                #colorscale='YlOrRd',
                reversescale=True,
                color=list_to_color(listevo[elt]),
                size=10,
                line=dict(width=2)))
        for node in pos:
            x = pos[node][0]
            y = pos[node][1]
            node_trace['x'] += tuple([x])
            node_trace['y'] += tuple([y])


        for node, adjacencies in enumerate(G.adjacency()):
            node_trace['marker']['color']+=tuple([len(adjacencies[1])])
            node_info ='Type: ' + listeprof[node]+' , # of friends: '+str(len(adjacencies[1]))
            node_trace['text']+=tuple([node_info])

        fig = go.Figure(data=[edge_trace, node_trace],
                     layout=go.Layout(
                        title='<br>Network graph made with Python',
                        titlefont=dict(size=16),
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="Population de personnes utilisant les réseaux sociaux et leur liaisons entre eux.",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
        if not os.path.exists('images'):
            os.mkdir('images')
        pio.write_image(fig, 'images/fig_'+str(elt)+'.png')
    print(listevo[-1])
    return py.iplot(fig, filename='networkx')
        
graph_propre_profils_transmit_groupe(mat,listep,10)

NameError: name 'os' is not defined

In [30]:

import sys
import datetime
import imageio
from os import listdir
from os.path import isfile, join
from IPython.display import Image
from IPython.display import display

def tri(fichier):
    """str->int
    récupère le numéro d'un fichier, utile pour la création du gif"""
    nbr=0
    nombres=set(str(i) for i in range(10))
    k=0
    boule=0
    for i in range(len(fichier)):
        if fichier[i] in nombres and boule==0:
            k=i
            boule=1
        elif fichier[i] in nombres:
            nbr=int(fichier[k:i+1])
    return nbr
            

def create_gif(duration):
    images = []
    filenames = sorted(["images/"+f for f in listdir("images") if isfile("images/"+f)], key=tri)
    output_file = 'Gif-%s.gif' % datetime.datetime.now().strftime('%Y-%M-%d-%H-%M-%S')
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(output_file, images, duration=duration)

    display(Image(url=output_file))




create_gif(0.5)